In [ ]:
#Find all place names <placeName> in <body> that match the list  <listPlace> in the header. Then, fetch the place xml:id and some other useful elements.
#From only Sloane catnums
#Ignore index
#FossilsCorals XML file
#9 <div2> with only 2 labels within
#some <div3> has <p> before <catnum>


In [ ]:
from lxml import etree
import re
import csv
import pandas as pd

In [ ]:
#to keep all namespaces correctly
#import re
#import xml.etree.ElementTree as ET

#ET.register_namespace('',"http://www.tei-c.org/ns/1.0")
#ET.register_namespace('ea',"http://www.enlightenmentarchitectures.org")
#mytree = ET.parse('/Users/deborahleem/Documents/1_PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL-KS_050321 (1).xml')

path = '/Users/deborahleem/Documents/1_PhD/Fossils1Ah_MASTER_DL24032021.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()
root = mytree

In [ ]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
            continue
        remove_element(e)

In [ ]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [ ]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [ ]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [ ]:
#which placeName goes with which ID

#viaf map shows viaf number with the corresponding person name
# This has nothing to do with the body text as mapping is done with <listPerson> and <listPlace> in the <Header>
idmap = {}
variant_id_map = {}
# go through master list place names
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}listPlace/{http://www.tei-c.org/ns/1.0}place'):
    id = x.attrib['{http://www.w3.org/XML/1998/namespace}id']
    # do find all on x object for placeName
    for name in x.findall('{http://www.tei-c.org/ns/1.0}placeName'):
        text = name.text
        # create a mapping of name to place_id        
        idmap[text.lower()] = '#' + id
        variant_id_map.setdefault(text.lower(), []).append(id)
        
#         idno = x.find("{*}idno")
#         # if idno is found add to viafmap
#         if idno is not None:
#             viafmap[text] = idno.text


In [ ]:
idmap

In [ ]:
# find mappings where there are multiple ids linked to a place name
{k:v for (k,v) in variant_id_map.items() if len(v) > 1}

In [ ]:
def lookup_place(name, x):
    """Lookup in our dictionary created from the master list to obtain place name ID"""
    #     Use the value from our dictionary variant_id_map
#     if name == 'Jamaica':
#         import pdb; pdb.set_trace()
    try:
        val = str(idmap[name.lower()])
#         val = val.replace('[', '').replace(']', '').replace('"', '').replace("'", '')
        print(name)
        print('Adding in:' + val)
#         return val
#         x.set('key', val)
        x.set('ref', val)
    except:
#           print('Issue adding in: ' + name)
        print('Issue adding in: ' + name + '\ncat num: ' + str('') + '\npage: ' + str('') + '\n')

In [ ]:
# We must reload the xml root.This is a duplicate of the first cell.
# This is because we don't want to remove things like add_rend.
# The lookups above are still in memory.
path = '/Users/deborahleem/Documents/1_PhD/Fossils1Ah_MASTER_DL24032021.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()
root = mytree

In [ ]:
place_csv_data = []

# labels = set()

# for div3 in root.iter("{*}div3"):
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}div3//{http://www.tei-c.org/ns/1.0}placeName'):
    name = re.sub(r'\s+', " ", ''.join(x.itertext()))
    name = re.sub(r'^\s+', '', name)
    name = re.sub(r'\s+$', '', name)   
    lookup_place(name, x)

In [ ]:
# save down the xml
doc.write("fossils_output_person_place_misc.xml", encoding='utf-8')

In [ ]:
from pprint import pprint as pp
# Place names from body
pp(place_csv_data)

In [ ]:
# Create the dataframe and populate based on the dictionary

df = pd.DataFrame({'div2_label': [], 'place': [], 'place_ids': [], 'catnum': []})
for row in place_csv_data:
    df = df.append({'div2_label': row['div2_label'], 'place': str(row['place']), 'place_ids': str(row['place_ids']), 'catnum': str(row['catnum'])}, ignore_index=True)
df.head()

In [ ]:
df[df.place_ids != '[None]'].to_csv('places_with_found_name_from_master_list.csv')


In [ ]:
len(df[df.place_ids != '[None]'])

In [ ]:
df[df.place_ids == '[None]'].to_csv('places_with_missing_name_from_master_list.csv')

In [ ]:
len(df[df.place_ids == '[None]'])